In [ ]:
def get_csv_results(IMAGE_FOLDER, OUTPUT_FOLDER, action_idx):
    print(IMAGE_FOLDER)
    OUTPUT = [] # Used to save the hand key information of all pictures in the entire current folder
    CSV_FILE = os.path.join(OUTPUT_FOLDER, f"{action_idx}.csv")
    IMAGE_FILES = os.listdir(IMAGE_FOLDER)
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
        for idx, file in enumerate(IMAGE_FILES):
            image_file = os.path.join(IMAGE_FOLDER, file)
            image = cv2.flip(cv2.imread(image_file), 1)
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            if not results.multi_handedness: # If the hand cannot be detected, skip it
                continue

            Left, Right = False, False  # Used to distinguish between left and right hands

            num_hands = len(results.multi_handedness)
            if num_hands == 2:
                Left, Right = True, True
            elif num_hands == 1:
                if results.multi_handedness[0].classification[0].label == "Left":
                    Left = True
                else:
                    Right = True

            output = {}
            output['Left'] = []
            output['Right'] = []
            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                if Left and idx == 0:
                    for data in hand_landmarks.landmark:
                        output["Left"] += [data.x, data.y, data.z]
                else:
                    for data in hand_landmarks.landmark:
                        output["Right"] += [data.x, data.y, data.z]
            OUTPUT.append(output)
    df = pd.DataFrame(OUTPUT)
    df.to_csv(CSV_FILE)

In [1]:
import os
import cv2

# Define folder name mapping
folder_name_mapping = {
    "Step_1": "1",
    "Step_2_Left": "2lt",
    "Step_2_Right": "2rt",
    "Step_3": "3",
    "Step_4_Left": "4lt",
    "Step_4_Right": "4rt",
    "Step_5_Left": "5lt",
    "Step_5_Right": "5rt",
    "Step_6_Left": "6lt",
    "Step_6_Right": "6rt",
    "Step_7_Left": "7lt",
    "Step_7_Right": "7rt",
}

def get_frame(file_path, save_folder):
    """Extract frames from a video file and save them as images."""
    video = cv2.VideoCapture(file_path)
    count = 0
    while True:
        success, frame = video.read()
        if not success:
            break
        save_path = os.path.join(save_folder, f'{count}.jpg')
        cv2.imwrite(save_path, frame)
        count += 1
    video.release()

def process_videos(dataset_path, output_path):
    """Process all .mp4 videos in dataset_path and save images in output_path."""
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith('.mp4'):  # Process only .mp4 files
                # Get the relative path of the folder
                relative_path = os.path.relpath(root, dataset_path)

                # Map the folder name to the new name
                folder_name = os.path.basename(relative_path)
                mapped_folder_name = folder_name_mapping.get(folder_name, folder_name)

                # Create the corresponding output folder
                save_folder = os.path.join(output_path, mapped_folder_name)
                os.makedirs(save_folder, exist_ok=True)

                # Get the full video file path
                file_path = os.path.join(root, file)

                # Extract frames and save them
                get_frame(file_path, save_folder)

                # Break after processing the first file
                #break

# Define paths
dataset_path = 'data/HandWashDataset'  # Path to the dataset folder
output_path = 'data/image'  # Path to the output folder

# Process videos
process_videos(dataset_path, output_path)


In [1]:
import os
import cv2
import mediapipe as mp
import pandas as pd

# Define the CSV generation function
def get_csv_results(IMAGE_FOLDER, OUTPUT_FOLDER, action_idx):
    print(f"Processing: {IMAGE_FOLDER}")
    OUTPUT = []  # Used to save the hand key information of all pictures in the current folder
    CSV_FILE = os.path.join(OUTPUT_FOLDER, f"{action_idx}.csv")
    IMAGE_FILES = sorted(os.listdir(IMAGE_FOLDER))  # Ensure sorted processing
    with mp.solutions.hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
        for idx, file in enumerate(IMAGE_FILES):
            image_file = os.path.join(IMAGE_FOLDER, file)
            #image = cv2.imread(image_file)
            image = cv2.flip(cv2.imread(image_file), 1) # 1:Rt&Lt 0:UP&Down -1:all
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            if not results.multi_handedness:  # If the hand cannot be detected, skip it
                continue

            Left, Right = False, False  # Used to distinguish between left and right hands

            num_hands = len(results.multi_handedness)
            if num_hands == 2:
                Left, Right = True, True
            elif num_hands == 1:
                if results.multi_handedness[0].classification[0].label == "Left":
                    Left = True
                else:
                    Right = True

            output = {}
            output['Left'] = []
            output['Right'] = []
            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                if Left and idx == 0:
                    for data in hand_landmarks.landmark:
                        output["Left"] += [data.x, data.y, data.z]
                else:
                    for data in hand_landmarks.landmark:
                        output["Right"] += [data.x, data.y, data.z]
            OUTPUT.append(output)
    df = pd.DataFrame(OUTPUT)
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)  # Ensure the output folder exists
    df.to_csv(CSV_FILE, index=False)

# Define paths
image_folder = 'data/image'  # Path to the folder containing extracted images
output_folder = 'data/landmark'  # Path to save the CSV files

# Generate CSVs for each folder in the /image directory
for folder_name in os.listdir(image_folder):
    folder_path = os.path.join(image_folder, folder_name)
    if os.path.isdir(folder_path):  # Ensure it's a directory
        get_csv_results(folder_path, output_folder, folder_name)


2025-01-25 14:52:02.453081: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-25 14:52:02.494601: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-25 14:52:02.496851: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-25 14:52:03.297867: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Processing: data/image/1


I0000 00:00:1737816724.176555   28568 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1737816724.187486   28665 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: AMD RENOIR (DRM 3.42.0, 5.15.0-130-generic, LLVM 12.0.0)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Processing: data/image/2lt


I0000 00:00:1737816768.456721   28568 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1737816768.458853   28717 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: AMD RENOIR (DRM 3.42.0, 5.15.0-130-generic, LLVM 12.0.0)


...

I0000 00:00:1737817123.469039   28568 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1737817123.470943   29818 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: AMD RENOIR (DRM 3.42.0, 5.15.0-130-generic, LLVM 12.0.0)


Processing: data/image/7lt


I0000 00:00:1737817161.669080   28568 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1737817161.670287   29845 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: AMD RENOIR (DRM 3.42.0, 5.15.0-130-generic, LLVM 12.0.0)


Processing: data/image/7rt


I0000 00:00:1737817198.929812   28568 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1737817198.931054   29874 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: AMD RENOIR (DRM 3.42.0, 5.15.0-130-generic, LLVM 12.0.0)
